In [42]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-storm-20/DataStorm2.0_Q1-v2.pdf
/kaggle/input/data-storm-20/Hotel-A-test.csv
/kaggle/input/data-storm-20/Hotel-A-train.csv
/kaggle/input/data-storm-20/Hotel-A-validation.csv
/kaggle/input/data-storm-20/Hotel-A-data-dictionary.csv


In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split,GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import warnings  
warnings.filterwarnings('ignore')



from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier,XGBRFClassifier
from sklearn.model_selection import cross_val_score

from collections import Counter

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve


In [44]:
train=pd.read_csv('/kaggle/input/data-storm-20/Hotel-A-train.csv')
vali=pd.read_csv('/kaggle/input/data-storm-20/Hotel-A-validation.csv')

test=pd.read_csv('/kaggle/input/data-storm-20/Hotel-A-test.csv')
test_set=test

In [45]:
train.head()

,Reservation-id,Gender,Age,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Expected_checkin,Expected_checkout,...,Meal_Type,Visted_Previously,Previous_Cancellations,Deposit_type,Booking_channel,Required_Car_Parking,Reservation_Status,Use_Promotion,Discount_Rate,Room_Rate
0,39428300,F,40,Latino,Grad,<25K,North,City Hotel,7/1/2015,7/2/2015,...,BB,No,No,No Deposit,Online,Yes,Check-In,Yes,10,218
1,77491756,F,49,Latino,Mid-School,50K -- 100K,East,City Hotel,7/1/2015,7/2/2015,...,BB,No,No,Refundable,Online,Yes,Check-In,No,0,185
2,73747291,F,42,caucasian,Grad,<25K,East,City Hotel,7/2/2015,7/6/2015,...,BB,No,No,No Deposit,Online,Yes,Check-In,No,0,119
3,67301739,M,25,African American,College,>100K,South,Airport Hotels,7/2/2015,7/3/2015,...,BB,No,No,Refundable,Agent,Yes,Check-In,Yes,5,144
4,77222321,F,62,Latino,High-School,25K --50K,East,Resort,7/3/2015,7/4/2015,...,BB,No,No,No Deposit,Direct,No,Check-In,Yes,10,242


In [46]:
from pandas_profiling import ProfileReport
profile = ProfileReport(train, title='Pandas Profiling Report', explorative=True)
#profile

In [47]:
print (train.dtypes)

Reservation-id             int64
Gender                    object
Age                        int64
Ethnicity                 object
Educational_Level         object
Income                    object
Country_region            object
Hotel_Type                object
Expected_checkin          object
Expected_checkout         object
Booking_date              object
Adults                     int64
Children                   int64
Babies                     int64
Meal_Type                 object
Visted_Previously         object
Previous_Cancellations    object
Deposit_type              object
Booking_channel           object
Required_Car_Parking      object
Reservation_Status        object
Use_Promotion             object
Discount_Rate              int64
Room_Rate                  int64
dtype: object


In [48]:
train['Income'].value_counts()

50K -- 100K    7936
25K --50K      7870
<25K           7739
>100K          3954
Name: Income, dtype: int64

In [49]:
new_I=[]

for i in train['Income']:
    
    if (i=='<25K'):
        new_I.append(1)
    elif (i=='25K --50K'):
        new_I.append(2)
    elif (i=='50K -- 100K'):
        new_I.append(3)
    else:
        new_I.append(4)
new_I=pd.DataFrame(new_I)
train.insert(6,'New_Income',new_I)
    
new_I=[]
for i in vali['Income']:
    
    if (i=='<25K'):
        new_I.append(1)
    elif (i=='25K --50K'):
        new_I.append(2)
    elif (i=='50K -- 100K'):
        new_I.append(3)
    else:
        new_I.append(4)
new_I=pd.DataFrame(new_I)
vali.insert(6,'New_Income',new_I)

new_I=[]
for i in test['Income']:
    
    if (i=='<25K'):
        new_I.append(1)
    elif (i=='25K --50K'):
        new_I.append(2)
    elif (i=='50K -- 100K'):
        new_I.append(3)
    else:
        new_I.append(4)
new_I=pd.DataFrame(new_I)
test.insert(6,'New_Income',new_I)

train=train.drop( 'Income',axis=1)
vali=vali.drop( 'Income',axis=1)
test=test.drop('Income', axis=1)

In [50]:
train['Educational_Level'].value_counts()

College        11052
High-School     5522
Mid-School      5481
Grad            5444
Name: Educational_Level, dtype: int64

In [51]:
new_E=[]

for i in train['Educational_Level']:
    
    if (i=='Mid-School'):
        new_E.append(1)
    elif (i=='High-School'):
        new_E.append(2)
    elif (i=='College'):
        new_E.append(3)
    else:
        new_E.append(4)

    
new_E=pd.DataFrame(new_E)
train.insert(5,'New_Education',new_E)



new_E=[]

for i in vali['Educational_Level']:
    
    if (i=='Mid-School'):
        new_E.append(1)
    elif (i=='High-School'):
        new_E.append(2)
    elif (i=='College'):
        new_E.append(3)
    else:
        new_E.append(4)

    
new_E=pd.DataFrame(new_E)
vali.insert(5,'New_Education',new_E)


new_E=[]

for i in test['Educational_Level']:
    
    if (i=='Mid-School'):
        new_E.append(1)
    elif (i=='High-School'):
        new_E.append(2)
    elif (i=='College'):
        new_E.append(3)
    else:
        new_E.append(4)

    
new_E=pd.DataFrame(new_E)
test.insert(5,'New_Education',new_E)

train=train.drop( 'Educational_Level',axis=1)
vali=vali.drop( 'Educational_Level',axis=1)
test=test.drop('Educational_Level',axis=1)

In [52]:
train['Reservation_Status'].value_counts()

Check-In    21240
Canceled     4134
No-Show      2125
Name: Reservation_Status, dtype: int64

In [53]:
Reservation=[]

for i in train['Reservation_Status']:
    
    if (i=='Check-In'):
        Reservation.append(1)
    elif (i=='Canceled'):
        Reservation.append(2)
    elif (i=='No-Show'):
        Reservation.append(3)

    
Reservation=pd.DataFrame(Reservation)
train.insert(18,'New_Reservation_Status',Reservation)



Reservation=[]

for i in vali['Reservation_Status']:
    
    if (i=='Check-In'):
        Reservation.append(1)
    elif (i=='Canceled'):
        Reservation.append(2)
    elif (i=='No-Show'):
        Reservation.append(3)
  

    
Reservation=pd.DataFrame(Reservation)
vali.insert(18,'New_Reservation_Status',Reservation)

In [54]:
train=train.drop( 'Reservation_Status',axis=1)
vali=vali.drop( 'Reservation_Status',axis=1)

In [55]:
train['New_Reservation_Status'].value_counts()

1    21240
2     4134
3     2125
Name: New_Reservation_Status, dtype: int64

In [56]:
target = 'New_Reservation_Status'
ID = 'Reservation-id'

In [57]:
y=train[target]
vali_target=vali[target]


In [58]:
train.head()

,Reservation-id,Gender,Age,Ethnicity,New_Education,New_Income,Country_region,Hotel_Type,Expected_checkin,Expected_checkout,...,Meal_Type,Visted_Previously,Previous_Cancellations,Deposit_type,New_Reservation_Status,Booking_channel,Required_Car_Parking,Use_Promotion,Discount_Rate,Room_Rate
0,39428300,F,40,Latino,4,1,North,City Hotel,7/1/2015,7/2/2015,...,BB,No,No,No Deposit,1,Online,Yes,Yes,10,218
1,77491756,F,49,Latino,1,3,East,City Hotel,7/1/2015,7/2/2015,...,BB,No,No,Refundable,1,Online,Yes,No,0,185
2,73747291,F,42,caucasian,4,1,East,City Hotel,7/2/2015,7/6/2015,...,BB,No,No,No Deposit,1,Online,Yes,No,0,119
3,67301739,M,25,African American,3,4,South,Airport Hotels,7/2/2015,7/3/2015,...,BB,No,No,Refundable,1,Agent,Yes,Yes,5,144
4,77222321,F,62,Latino,2,2,East,Resort,7/3/2015,7/4/2015,...,BB,No,No,No Deposit,1,Direct,No,Yes,10,242


In [59]:
train=train.drop([target,ID,'Booking_date','Expected_checkin', 'Expected_checkout'],axis=1)
vali=vali.drop([target,ID,'Booking_date','Expected_checkin', 'Expected_checkout'],axis=1)
test=test.drop([ID,'Booking_date','Expected_checkin', 'Expected_checkout'],axis=1)

In [60]:
family=train[['Adults','Children','Babies']].sum(axis=1)
train.insert(6,'Family',family)

family=vali[['Adults','Children','Babies']].sum(axis=1)
vali.insert(6,'Family',family)

family=test[['Adults','Children','Babies']].sum(axis=1)
test.insert(6,'Family',family)

train=train.drop(['Adults','Children','Babies'],axis=1)
vali=vali.drop(['Adults','Children','Babies'],axis=1)
test=test.drop(['Adults','Children','Babies'],axis=1)

In [61]:
train.head()

,Gender,Age,Ethnicity,New_Education,New_Income,Country_region,Family,Hotel_Type,Meal_Type,Visted_Previously,Previous_Cancellations,Deposit_type,Booking_channel,Required_Car_Parking,Use_Promotion,Discount_Rate,Room_Rate
0,F,40,Latino,4,1,North,4,City Hotel,BB,No,No,No Deposit,Online,Yes,Yes,10,218
1,F,49,Latino,1,3,East,6,City Hotel,BB,No,No,Refundable,Online,Yes,No,0,185
2,F,42,caucasian,4,1,East,6,City Hotel,BB,No,No,No Deposit,Online,Yes,No,0,119
3,M,25,African American,3,4,South,7,Airport Hotels,BB,No,No,Refundable,Agent,Yes,Yes,5,144
4,F,62,Latino,2,2,East,2,Resort,BB,No,No,No Deposit,Direct,No,Yes,10,242


In [62]:
numerical_column=[col for col in train.columns if train[col].dtype in ['int64','float64']]

# find the categorical column in the data
categorical_column=list(set(train.columns)-set(numerical_column))

print("Numerical columns",numerical_column)


Numerical columns ['Age', 'New_Education', 'New_Income', 'Family', 'Discount_Rate', 'Room_Rate']


In [63]:
print('Categorical columns',categorical_column)

Categorical columns ['Gender', 'Booking_channel', 'Ethnicity', 'Country_region', 'Required_Car_Parking', 'Hotel_Type', 'Deposit_type', 'Use_Promotion', 'Previous_Cancellations', 'Visted_Previously', 'Meal_Type']


In [64]:
#Label=[ 'Visted_Previously', 'Previous_Cancellations',  'Use_Promotion',  'Required_Car_Parking',  'Gender']
#categorical_column=list(set(categorical_column)-set(Label))
#categorical_column

In [65]:
'''import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline



class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
'''Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.'''
'''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)
    
#train=MultiColumnLabelEncoder(columns = [ 'Visted_Previously', 'Previous_Cancellations',  'Use_Promotion',  'Required_Car_Parking',  'Gender']).fit_transform(train)
#vali=MultiColumnLabelEncoder(columns = [ 'Visted_Previously', 'Previous_Cancellations',  'Use_Promotion',  'Required_Car_Parking',  'Gender']).fit_transform(vali)

'''

"\n        output = X.copy()\n        if self.columns is not None:\n            for col in self.columns:\n                output[col] = LabelEncoder().fit_transform(output[col])\n        else:\n            for colname,col in output.iteritems():\n                output[colname] = LabelEncoder().fit_transform(col)\n        return output\n\n    def fit_transform(self,X,y=None):\n        return self.fit(X,y).transform(X)\n    \n#train=MultiColumnLabelEncoder(columns = [ 'Visted_Previously', 'Previous_Cancellations',  'Use_Promotion',  'Required_Car_Parking',  'Gender']).fit_transform(train)\n#vali=MultiColumnLabelEncoder(columns = [ 'Visted_Previously', 'Previous_Cancellations',  'Use_Promotion',  'Required_Car_Parking',  'Gender']).fit_transform(vali)\n\n"

In [66]:
train.head()

,Gender,Age,Ethnicity,New_Education,New_Income,Country_region,Family,Hotel_Type,Meal_Type,Visted_Previously,Previous_Cancellations,Deposit_type,Booking_channel,Required_Car_Parking,Use_Promotion,Discount_Rate,Room_Rate
0,F,40,Latino,4,1,North,4,City Hotel,BB,No,No,No Deposit,Online,Yes,Yes,10,218
1,F,49,Latino,1,3,East,6,City Hotel,BB,No,No,Refundable,Online,Yes,No,0,185
2,F,42,caucasian,4,1,East,6,City Hotel,BB,No,No,No Deposit,Online,Yes,No,0,119
3,M,25,African American,3,4,South,7,Airport Hotels,BB,No,No,Refundable,Agent,Yes,Yes,5,144
4,F,62,Latino,2,2,East,2,Resort,BB,No,No,No Deposit,Direct,No,Yes,10,242


In [67]:
test.head()

,Gender,Age,Ethnicity,New_Education,New_Income,Country_region,Family,Hotel_Type,Meal_Type,Visted_Previously,Previous_Cancellations,Deposit_type,Booking_channel,Required_Car_Parking,Use_Promotion,Discount_Rate,Room_Rate
0,F,52,Latino,4,2,South,6,City Hotel,HB,No,No,No Deposit,Direct,Yes,Yes,10,153
1,F,47,Latino,4,2,East,3,Airport Hotels,FB,No,No,No Deposit,Online,No,No,0,210
2,F,28,Asian American,4,1,East,4,City Hotel,BB,No,No,No Deposit,Agent,No,Yes,5,117
3,F,65,caucasian,2,2,South,6,Airport Hotels,FB,No,No,No Deposit,Online,Yes,Yes,10,107
4,M,45,African American,3,2,South,4,City Hotel,BB,No,No,Refundable,Agent,No,No,0,119


In [68]:
encoding_instance = OneHotEncoder(handle_unknown='ignore')
train_OneHot = pd.DataFrame(encoding_instance.fit_transform(train[categorical_column]).toarray())

encoding_instance = OneHotEncoder(handle_unknown='ignore')
vali_OneHot = pd.DataFrame(encoding_instance.fit_transform(vali[categorical_column]).toarray())

encoding_instance = OneHotEncoder(handle_unknown='ignore')
test_OneHot = pd.DataFrame(encoding_instance.fit_transform(test[categorical_column]).toarray())



In [69]:
train_data = pd.concat([train,train_OneHot],axis=1)
vali_data = pd.concat([vali,vali_OneHot],axis=1)
test_data=pd.concat([test,test_OneHot], axis=1)

In [70]:
X = train_data.drop( categorical_column,axis=1)

In [71]:
train_data.head()

,Gender,Age,Ethnicity,New_Education,New_Income,Country_region,Family,Hotel_Type,Meal_Type,Visted_Previously,...,20,21,22,23,24,25,26,27,28,29
0,F,40,Latino,4,1,North,4,City Hotel,BB,No,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,F,49,Latino,1,3,East,6,City Hotel,BB,No,...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
2,F,42,caucasian,4,1,East,6,City Hotel,BB,No,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,M,25,African American,3,4,South,7,Airport Hotels,BB,No,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
4,F,62,Latino,2,2,East,2,Resort,BB,No,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0


In [72]:
X.head()

,Age,New_Education,New_Income,Family,Discount_Rate,Room_Rate,0,1,2,3,...,20,21,22,23,24,25,26,27,28,29
0,40,4,1,4,10,218,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,49,1,3,6,0,185,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
2,42,4,1,6,0,119,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,25,3,4,7,5,144,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
4,62,2,2,2,10,242,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0


In [73]:

scaler = StandardScaler().fit(X)
X = pd.DataFrame(scaler.transform(X))


vali = vali_data.drop(categorical_column,axis=1)
scaler = StandardScaler().fit(vali)
vali = pd.DataFrame(scaler.transform(vali))


test=test_data.drop(categorical_column,axis=1)
scaler=StandardScaler().fit(test)
test=pd.DataFrame(scaler.transform(test))

In [74]:
#xgboost_model = XGBRFClassifier(n_estimators=500,learning_rate=0.01)
#scores = cross_val_score(xgboost_model, X, y, cv=5, scoring='accuracy')
#print(scores)
#print (np.mean(scores))

'''
grid_data = {'n_estimators': [200,300,500],'learning_rate': [0.1,0.01,0.05]}

grid_search = GridSearchCV(XGBClassifier(),grid_data,refit=True,verbose=3,cv=3)
grid_search.fit(X,y)

print(grid_search.best_params_)
print(grid_search.best_score_)
'''

"\ngrid_data = {'n_estimators': [200,300,500],'learning_rate': [0.1,0.01,0.05]}\n\ngrid_search = GridSearchCV(XGBClassifier(),grid_data,refit=True,verbose=3,cv=3)\ngrid_search.fit(X,y)\n\nprint(grid_search.best_params_)\nprint(grid_search.best_score_)\n"

In [75]:
'''DTC = DecisionTreeClassifier()

adaDTC = AdaBoostClassifier(DTC, random_state=7)

ada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "algorithm" : ["SAMME","SAMME.R"],
              "n_estimators" :[1,2],
              "learning_rate":  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3,1.5]}

gsadaDTC = GridSearchCV(adaDTC,param_grid = ada_param_grid, cv=5, scoring="accuracy", n_jobs= 4, verbose = 1)

gsadaDTC.fit(X,y)

gsadaDTC.best_estimator_'''

'DTC = DecisionTreeClassifier()\n\nadaDTC = AdaBoostClassifier(DTC, random_state=7)\n\nada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"],\n              "base_estimator__splitter" :   ["best", "random"],\n              "algorithm" : ["SAMME","SAMME.R"],\n              "n_estimators" :[1,2],\n              "learning_rate":  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3,1.5]}\n\ngsadaDTC = GridSearchCV(adaDTC,param_grid = ada_param_grid, cv=5, scoring="accuracy", n_jobs= 4, verbose = 1)\n\ngsadaDTC.fit(X,y)\n\ngsadaDTC.best_estimator_'

In [76]:
#print(gsadaDTC.best_params_)
#print(gsadaDTC.best_score_)

In [77]:
#gsadaDTC.best_score_

In [78]:
#baseline result

classifiers=[XGBClassifier(n_estimators=200,learning_rate=0.05, max_depth=10)] 



for model in classifiers:
    model.fit(X,y)

    pred = model.predict(vali)
    print(model)
    print('')
    
    print(classification_report(vali_target,pred))
    
    print ('-------------------------------------')
    print('')
    
    print(confusion_matrix(vali_target,pred))

[15:59:45] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

              precision    recall  f1-score   support

           1       0.59      0.98      0.73      1610
           2    

In [79]:

from sklearn.datasets import make_classification
from collections import Counter

counter = Counter(vali_target)

In [80]:
counter

Counter({3: 398, 2: 741, 1: 1610})

In [81]:
'''from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier

model = XGBClassifier()
# define grid
weights = [1, 10, 25, 50, 75, 99, 100, 1000]
param_grid = dict(scale_pos_weight=weights)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))'''

'from numpy import mean\nfrom sklearn.datasets import make_classification\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.model_selection import RepeatedStratifiedKFold\nfrom xgboost import XGBClassifier\n\nmodel = XGBClassifier()\n# define grid\nweights = [1, 10, 25, 50, 75, 99, 100, 1000]\nparam_grid = dict(scale_pos_weight=weights)\n# define evaluation procedure\ncv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)\n# define grid search\ngrid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=\'roc_auc\')\n# execute the grid search\ngrid_result = grid.fit(X, y)\n# report the best configuration\nprint("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))\n# report all configurations\nmeans = grid_result.cv_results_[\'mean_test_score\']\nstds = grid_result.cv_results_[\'std_test_score\']\nparams = grid_result.cv_results_[\'params\']\nfor mean, stdev, param in zip(means, stds, params):\n    print("%f (

In [82]:
#class balancing 1

def CreateBalancedSampleWeights(y_train, largest_class_weight_coef):
    classes = y_train.unique()
    classes.sort()
    class_samples = np.bincount(y_train)
    total_samples = class_samples.sum()
    n_classes = len(class_samples)
    weights = total_samples / (n_classes * class_samples * 1.0)
    class_weight_dict = {key : value for (key, value) in zip(classes, weights)}
    class_weight_dict[classes[1]] = class_weight_dict[classes[1]] * largest_class_weight_coef
    sample_weights = [class_weight_dict[y] for y in y_train]

    return sample_weights



largest_class_weight_coef = max(y.value_counts().values)/y.shape[0]
 #pass y_train as numpy array
weight = CreateBalancedSampleWeights(y, largest_class_weight_coef)



In [83]:
xg=XGBClassifier(n_estimators=200,learning_rate=0.05, weights=weight, max_depth=10)

xg.fit(X,y)

pred = xg.predict(vali)

print('')
    
print(classification_report(vali_target,pred))

print ('-------------------------------------')
print('')
    
print(confusion_matrix(vali_target,pred))

[16:00:11] WARNING: ../src/learner.cc:541: 
Parameters: { weights } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:00:11] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

              precision    recall  f1-score   support

           1       0.59      0.98      0.73      1610
           2       0.27      0.02      0.04       741
           3       0.00      0.00      0.00       398

    accuracy                           0.58      2749
   macro avg       0.29      0.33      0.26      2749
weighted avg       0.42      0.58      0.44      2749

-------------------------------------


In [84]:
xg=XGBClassifier(n_estimators=200,learning_rate=0.05, max_depth=10)

xg.fit(X,y,sample_weight=weight)

pred = xg.predict(vali)

print('')
    
print(classification_report(vali_target,pred))

print ('-------------------------------------')
print('')
    
print(confusion_matrix(vali_target,pred))

[16:00:36] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

              precision    recall  f1-score   support

           1       0.59      1.00      0.74      1610
           2       0.00      0.00      0.00       741
           3       0.00      0.00      0.00       398

    accuracy                           0.59      2749
   macro avg       0.20      0.33      0.25      2749
weighted avg       0.34      0.59      0.43      2749

-------------------------------------

[[1610    0    0]
 [ 741    0    0]
 [ 398    0    0]]


In [85]:
from sklearn.utils.class_weight import compute_sample_weight

w=compute_sample_weight(class_weight='balanced', y=y)



In [86]:
#grid_data = {'n_estimators': [200,300,500],'learning_rate': [0.1,0.01,0.05]}

#grid_search = GridSearchCV(XGBRFClassifier(),grid_data,refit=True,verbose=3,cv=3)
#grid_search.fit(X,y,sample_weight = w)



In [87]:
#print(grid_search.best_params_)
#print(grid_search.best_score_)

In [88]:
xg=XGBClassifier(n_estimators=500,learning_rate=0.001, max_depth=20)
xg.fit(X, y,sample_weight = w)
pred = xg.predict(vali)
print(classification_report(vali_target,pred))
    
print ('-------------------------------------')
print('')
    
print(confusion_matrix(vali_target,pred))

[16:00:39] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           1       0.60      0.67      0.63      1610
           2       0.28      0.21      0.24       741
           3       0.15      0.14      0.14       398

    accuracy                           0.47      2749
   macro avg       0.34      0.34      0.34      2749
weighted avg       0.45      0.47      0.45      2749

-------------------------------------

[[1077  319  214]
 [ 479  158  104]
 [ 247   97   54]]


In [89]:
from sklearn.metrics import f1_score

a=f1_score(vali_target,pred, average='macro')
b=f1_score(vali_target,pred, average='micro')
c=f1_score(vali_target,pred, average='weighted')


In [90]:
#grid_data = {'n_estimators': [200,300,500,600],'learning_rate': [0.1,0.01,0.05]}

#grid_search = GridSearchCV(AdaBoostClassifier(),grid_data,refit=True,verbose=3,cv=3)
#grid_search.fit(X,y,sample_weight = w)



In [91]:
#print(grid_search.best_params_)
#print(grid_search.best_score_)

In [92]:
sss

NameError: name 'sss' is not defined

In [ ]:
cc=[RandomForestClassifier(n_estimators=500),LogisticRegression(),LinearSVC(),SVC(), 
    RandomForestClassifier(),GaussianNB(),Perceptron(),SGDClassifier(),DecisionTreeClassifier(),
   AdaBoostClassifier(n_estimators=100, random_state=0)]

for model in cc:
    model.fit(X, y,sample_weight = w)
    pred = model.predict(vali)
    print(str(model))
    print('')
    print(classification_report(vali_target,pred))

   
    print('')

    print(confusion_matrix(vali_target,pred))
    print ('-------------------------------------')
    print ('-------------------------------------')
    
#test=test.to_numpy()
#test_prediction=model.predict(test)
#output = pd.DataFrame({'PassengerId': Id,
  #                     'Survived': test_prediction})
#output.to_csv('submission.csv', index=False)

In [93]:
model=DecisionTreeClassifier( random_state=0)

model.fit(X, y,sample_weight = w)
pred = model.predict(vali)

print(classification_report(vali_target,pred))

print('')

print(confusion_matrix(vali_target,pred))

              precision    recall  f1-score   support

           1       0.59      0.73      0.65      1610
           2       0.28      0.19      0.23       741
           3       0.14      0.10      0.11       398

    accuracy                           0.49      2749
   macro avg       0.34      0.34      0.33      2749
weighted avg       0.44      0.49      0.46      2749


[[1172  280  158]
 [ 527  140   74]
 [ 287   73   38]]


In [97]:
test_set.head()

,Reservation-id,Gender,Age,Ethnicity,Educational_Level,Income,New_Income,Country_region,Hotel_Type,Expected_checkin,...,Babies,Meal_Type,Visted_Previously,Previous_Cancellations,Deposit_type,Booking_channel,Required_Car_Parking,Use_Promotion,Discount_Rate,Room_Rate
0,62931593,F,52,Latino,Grad,25K --50K,2,South,City Hotel,11/18/2016,...,0,HB,No,No,No Deposit,Direct,Yes,Yes,10,153
1,70586099,F,47,Latino,Grad,25K --50K,2,East,Airport Hotels,11/18/2016,...,0,FB,No,No,No Deposit,Online,No,No,0,210
2,4230648,F,28,Asian American,Grad,<25K,1,East,City Hotel,4/28/2017,...,0,BB,No,No,No Deposit,Agent,No,Yes,5,117
3,25192322,F,65,caucasian,High-School,25K --50K,2,South,Airport Hotels,11/18/2016,...,2,FB,No,No,No Deposit,Online,Yes,Yes,10,107
4,80931528,M,45,African American,College,25K --50K,2,South,City Hotel,11/18/2016,...,0,BB,No,No,Refundable,Agent,No,No,0,119


In [100]:
target

'New_Reservation_Status'

In [96]:
#test=test.to_numpy()
test_prediction=model.predict(test)
output = pd.DataFrame({'Reservation-id': test_set[ID],
                  'Reservation_Status': test_prediction})
output.to_csv('submission.csv', index=False)

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    """Generate a simple plot of the test and training learning curve"""
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

g = plot_learning_curve(grid_search.best_estimator_,"RF mearning curves",X,y,cv=5)



In [ ]:
DTC = DecisionTreeClassifier()

adaDTC = AdaBoostClassifier(DTC, random_state=7)

ada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "algorithm" : ["SAMME","SAMME.R"],
              "n_estimators" :[1,2],
              "learning_rate":  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3,1.5]}

gsadaDTC = GridSearchCV(adaDTC,param_grid = ada_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsadaDTC.fit(X_train,Y_train)

ada_best = gsadaDTC.best_estimator_